In [3]:
from os import environ as ENV
from time import time as now
import json
import qbraid

import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append(ENV['PWD'])

from should_be_stdlib import *

In [2]:
# number of shots to use
SHOTS = 1000

# Load circuits

In [32]:
ang_circuits_df =     pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_angle_embed.xlsx', index_col=0)
amp_circuits_df =     pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed.xlsx', index_col=0)
amp_qft_circuits_df = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_qft.xlsx', index_col=0)
[x.shape for x in [ang_circuits_df, amp_circuits_df, amp_qft_circuits_df]]

[(2926, 3), (2926, 3), (2926, 3)]

In [33]:
# transpile circuits ahead of time
# if IonQ-qiskit is installed, the ionq job submission expects qiskit circuits instead of the native ionq format!!
# but qiskit transpiler is much faster so it's fine
import qbraid
amp_qft_circuits  = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_qft_circuits_df['qasm2'])]
for i,c in enumerate(amp_qft_circuits): c.name = f'amp_qft_c{i}'

amp_circuits      = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_circuits_df['qasm2'])]
for i,c in enumerate(amp_circuits): c.name = f'amp_c{i}'

ang_circuits      = [qbraid.transpile(c, 'qiskit') for c in tqdm(ang_circuits_df['qasm2'])]
for i,c in enumerate(ang_circuits): c.name = f'ang_c{i}'

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

# IonQ

## Connect to IonQ API

In [27]:
from provider_ionq import IONQ
ionq = IONQ(
    ionq_api_key := open(ENV['HOME'] + "/work/api-keys/IONQ_API_KEY").readline().strip()
)
ionq.printqpus()

ionq IonQDevice('qpu.harmony') 11qb DeviceStatus.RETIRED now!
ionq IonQDevice('qpu.aria-1') 25qb DeviceStatus.ONLINE queue 219d18h42m54s
ionq IonQDevice('qpu.aria-2') 25qb DeviceStatus.UNAVAILABLE queue 2288d10h2m14s
ionq IonQDevice('qpu.forte-1') 36qb DeviceStatus.UNAVAILABLE queue 65d2h8m35s
ionq IonQDevice('qpu.forte-enterprise-1') 36qb DeviceStatus.ONLINE queue 544d8h45m46s
ionq IonQDevice('qpu.forte-enterprise-2') 36qb DeviceStatus.UNAVAILABLE now!
ionq IonQDevice('simulator') 29qb DeviceStatus.ONLINE now!


In [6]:
ionq_qpu = ionq.provider.get_device('qpu.forte-enterprise-1')
ionq_qpu.status()

<DeviceStatus.ONLINE: 'online'>

## Run circuits

In principle, one could run these circuits. However, the estimated cost of IonQ Forte is very high, so we chose not to run them to save money.

In [7]:
import warnings
warnings.filterwarnings('ignore', message='No gate definition for.*') # hide transpiler warnings for ionq

In [8]:
# this submits a preflight job, no costs are incurred
def test_run(*args, **kwargs):
    kwargs['preflight'] = True
    ionq_job = ionq_qpu.run(*args, **kwargs)
    print(f'Estimating cost of {kwargs["name"]}')
    ionq_job.wait_for_final_state()
    metadata = ionq_job.metadata()
    cost_usd = metadata["cost_usd"]
    predicted_execution_time = metadata["predicted_execution_time"]
    print(f'${cost_usd}, {predicted_execution_time / 1000} seconds')

In [9]:
# test run our jobs
if verify_run():
    test_run(amp_qft_circuits[0], preflight=True, shots=SHOTS, name="Amp QFT",           error_mitigation={'debias': True})
    test_run(amp_qft_circuits[0], preflight=True, shots=SHOTS, name="Amp QFT no QEM",    error_mitigation={'debias': False})
    # test_run(amp_circuits[0],     preflight=True, shots=SHOTS, name="Amp no QFT",        error_mitigation={'debias': True})
    # test_run(amp_circuits[0],     preflight=True, shots=SHOTS, name="Amp no QFT no QEM", error_mitigation={'debias': False})
    test_run(ang_circuits[0],     preflight=True, shots=SHOTS, name="Angle",             error_mitigation={'debias': True})
    test_run(ang_circuits[0],     preflight=True, shots=SHOTS, name="Angle no QEM",      error_mitigation={'debias': False})

You must type "REALLY" to run this because it may cost a lot of money!!! >>>  no


By not typing "REALLY", you did not run this code


In [10]:
# run for real
if verify_run():
    ionq_qpu.run(amp_qft_circuits[0], shots=SHOTS, name="REAL Amp QFT",           error_mitigation={'debias': True})
    ionq_qpu.run(amp_qft_circuits[0], shots=SHOTS, name="REAL Amp QFT no QEM",    error_mitigation={'debias': False})
    # ionq_qpu.run(amp_circuits[0],     shots=SHOTS, name="REAL Amp no QFT",        error_mitigation={'debias': True})
    # ionq_qpu.run(amp_circuits[0],     shots=SHOTS, name="REAL Amp no QFT no QEM", error_mitigation={'debias': False})
    ionq_qpu.run(ang_circuits[0],     shots=SHOTS, name="REAL Angle",             error_mitigation={'debias': True})
    ionq_qpu.run(ang_circuits[0],     shots=SHOTS, name="REAL Angle no QEM",      error_mitigation={'debias': False})

You must type "REALLY" to run this because it may cost a lot of money!!! >>>  no


By not typing "REALLY", you did not run this code


In [4]:
import pandas as pd
amp_qft_ddd_circuits_df = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_qft_ddd.xlsx', index_col=0)
amp_qft_ddd_circuits  = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_qft_ddd_circuits_df['qasm2_ddd'])]
for i,c in enumerate(amp_qft_ddd_circuits): c.name = f'amp_qft_ddd_c{i}'

  0%|          | 0/2926 [00:00<?, ?it/s]

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [7]:
amp_qft_ddd_circuits[0].draw()

┌───┐┌────────────┐                                                   »
q_0: ┤ H ├┤ Ry(1.4733) ├──■───────────────────■─────────────────────────■──»
     ├───┤├────────────┤┌─┴─┐┌─────────────┐┌─┴─┐                       │  »
q_1: ┤ H ├┤ Ry(1.7653) ├┤ X ├┤ Ry(0.18643) ├┤ X ├──■────────────────────┼──»
     ├───┤├────────────┤└───┘└─────────────┘└───┘┌─┴─┐┌──────────────┐┌─┴─┐»
q_2: ┤ H ├┤ Ry(1.6276) ├─────────────────────────┤ X ├┤ Ry(0.055457) ├┤ X ├»
     ├───┤├────────────┤┌───┐     ┌───┐     ┌───┐├───┤└────┬───┬─────┘├───┤»
q_3: ┤ H ├┤ Ry(1.5993) ├┤ I ├─────┤ I ├─────┤ X ├┤ I ├─────┤ Y ├──────┤ I ├»
     └───┘└────────────┘└───┘     └───┘     └───┘└───┘     └───┘      └───┘»
c: 4/══════════════════════════════════════════════════════════════════════»
                                                                           »
«                                                   ┌───┐     ┌───┐      ┌───┐»
«q_0: ───────────────────────────────────────────■──┤ I ├─────┤ X ├──────┤ Y ├»
«                              ┌───┐      ┌───┐  │  ├───┤     ├───┤      └───┘»
«q_1: ─────────────────■───────┤ X ├──────┤ Y ├──┼──┤ X ├─────┤ Y ├────────■──»
«     ┌─────────────┐┌─┴─┐┌────┴───┴─────┐└───┘┌─┴─┐└───┘     └───┘        │  »
«q_2: ┤ Ry(0.15653) ├┤ X ├┤ Ry(0.036382) ├─────┤ X ├──■────────────────────┼──»
«     └────┬───┬────┘├───┤└────┬───┬─────┘┌───┐└───┘┌─┴─┐┌──────────────┐┌─┴─┐»
«q_3: ─────┤ X ├─────┤ I ├─────┤ Y ├──────┤ I ├─────┤ X ├┤ Ry(-0.33217) ├┤ X ├»
«          └───┘     └───┘     └───┘      └───┘     └───┘└──────────────┘└───┘»
«c: 4/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«           ┌───┐      ┌───┐     ┌───┐                     ┌───┐      ┌───┐»
«q_0: ──────┤ X ├──────┤ Y ├─────┤ I ├─────────────■───────┤ I ├──────┤ I ├»
«           ├───┤      ├───┤     ├───┤      ┌───┐  │       ├───┤      ├───┤»
«q_1: ──────┤ I ├──────┤ I ├─────┤ X ├──────┤ Y ├──┼───────┤ X ├──────┤ Y ├»
«           └───┘      └───┘     └───┘      └───┘  │       └───┘      └───┘»
«q_2: ───────────────────■─────────────────────────┼────────────────────■──»
«     ┌───────────────┐┌─┴─┐┌──────────────┐     ┌─┴─┐┌──────────────┐┌─┴─┐»
«q_3: ┤ Ry(-0.084059) ├┤ X ├┤ Ry(0.019854) ├─────┤ X ├┤ Ry(0.070394) ├┤ X ├»
«     └───────────────┘└───┘└──────────────┘     └───┘└──────────────┘└───┘»
«c: 4/═════════════════════════════════════════════════════════════════════»
«                                                                          »
«          ┌───┐      ┌───┐     ┌───┐      ┌───┐     ┌───┐                »
«q_0: ─────┤ X ├──────┤ Y ├─────┤ X ├──────┤ Y ├─────┤ I ├─────────────■──»
«          ├───┤      └───┘┌────┴───┴────┐ ├───┤     ├───┤      ┌───┐  │  »
«q_1: ─────┤ I ├────────■──┤ U3(0,0,π/4) ├─┤ X ├─────┤ Y ├──────┤ X ├──┼──»
«          └───┘        │  └─────────────┘ └───┘┌────┴───┴────┐ ├───┤  │  »
«q_2: ──────────────────┼────────────────────■──┤ U3(0,0,π/8) ├─┤ I ├──┼──»
«     ┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐├─────────────┴┐└───┘┌─┴─┐»
«q_3: ┤ Ry(0.025925) ├┤ X ├┤ Ry(0.017178) ├┤ X ├┤ Ry(0.049322) ├─────┤ X ├»
«     └──────────────┘└───┘└──────────────┘└───┘└──────────────┘     └───┘»
«c: 4/════════════════════════════════════════════════════════════════════»
«                                                                         »
«          ┌───┐      ┌───┐┌──────────────┐┌───┐┌─────────────┐     ┌───┐»
«q_0: ─────┤ H ├──────┤ X ├┤ U3(0,0,7π/4) ├┤ X ├┤ U3(0,0,π/4) ├─────┤ X ├»
«          ├───┤      └─┬─┘└──────────────┘└─┬─┘└────┬───┬────┘┌───┐└─┬─┘»
«q_1: ─────┤ Y ├────────■────────────────────■───────┤ H ├─────┤ X ├──┼──»
«          ├───┤      ┌───┐     ┌───┐      ┌───┐     ├───┤     └───┘  │  »
«q_2: ─────┤ X ├──────┤ Y ├─────┤ X ├──────┤ Y ├─────┤ I ├────────────■──»
«     ┌────┴───┴─────┐├───┤     ├───┤      ├───┤     ├───┤     ┌───┐┌───┐»
«q_3: ┤ U3(0,0,π/16) ├┤ I ├─────┤ I ├──────┤ X ├─────┤ Y ├─────┤ X ├┤ 